In [1]:
import k3d
import numpy as np
import trimesh
from FastSpatial import *
from scipy.stats import special_ortho_group
import ipywidgets as widgets

import os


# Load a point cloud from a file (e.g., .ply, .xyz, .pcd)
file_name = '771.ply'
point_cloud_gt = trimesh.load(f"./examples/ground_truth/{file_name}")
point_cloud_1 = trimesh.load(f"./examples/model1/{file_name}")
point_cloud_2 = trimesh.load(f"./examples/model2/{file_name}")

points_gt = np.array(point_cloud_gt.vertices, dtype=np.float64)
points_1 = np.array(point_cloud_1.vertices, dtype=np.float64)
points_2 = np.array(point_cloud_2.vertices, dtype=np.float64)

# apply some random rotation to the point clouds
points_gt = points_gt @ special_ortho_group.rvs(3).T
points_1 = points_1 @ special_ortho_group.rvs(3).T
points_2 = points_2 @ special_ortho_group.rvs(3).T

pc_gt = k3d.points(points_gt, point_size=0.005 * (points_gt.max() - points_gt.min()), color=0x00ff88)
pc_1 = k3d.points(points_1,  point_size=0.005 * (points_1.max() - points_1.min()), color=0xff0000)
pc_2 = k3d.points(points_2, point_size=0.005 * (points_2.max() - points_2.min()), color=0x0000ff)

plot = k3d.plot(grid_visible=False)
plot += pc_gt
plot += pc_1
plot += pc_2
plot.display()

/home/amin/miniforge3/envs/spatial/lib/python3.10/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Output()

In [2]:
# normalize the point clouds and visualize again
points_gt_normalized = procrustes(points_gt[None])[0][0]
points_1_normalized = procrustes(points_1[None])[0][0]
points_2_normalized = procrustes(points_2[None])[0][0]

principal_gt = principal_axes(points_gt_normalized[None])[0]
principal_1 = principal_axes(points_1_normalized[None])[0]
principal_2 = principal_axes(points_2_normalized[None])[0]

plot = k3d.plot(grid_visible=False)
plot += k3d.points(points_gt_normalized, point_size=0.05, color=0x00ff88)
plot += k3d.points(points_1_normalized, point_size=0.05, color=0xff0000)
plot += k3d.points(points_2_normalized, point_size=0.05, color=0x0000ff)

plot.display()

Output()

In [3]:
# plot each point cloud with its principal axes
plot1 = k3d.plot(grid_visible=False)
plot1 += k3d.points(points_gt_normalized, point_size=0.05, color=0x00ff88)

for i in range(3):
    plot1 += k3d.line([np.zeros(3), principal_gt[i]*4], color=0x00ff88, width=0.05)
plot1.layout.width = '30%'
# plot.display()

plot2 = k3d.plot(grid_visible=False)
plot2 += k3d.points(points_1_normalized, point_size=0.05, color=0xff0000)

for i in range(3):
    plot2 += k3d.line([np.zeros(3), principal_1[i]*4], color=0xff0000, width=0.05)
plot2.layout.width = '30%'

plot3 = k3d.plot(grid_visible=False)
plot3 += k3d.points(points_2_normalized, point_size=0.05, color=0x0000ff)

for i in range(3):
    plot3 += k3d.line([np.zeros(3), principal_2[i]*4], color=0x0000ff, width=0.05)
plot3.layout.width = '30%'

widgets.HBox([plot1, plot2, plot3])

In [4]:
# align the point clouds and visualize again
points_1_aligned, normalized_cd_1 = align_clouds(points_1, points_gt, normalize=True)
points_2_aligned, normalized_cd_2 = align_clouds(points_2, points_gt, normalize=True)

point_size = 0.01 * (points_gt.max() - points_gt.min())

plot1 = k3d.plot(grid_visible=False)
plot1 += k3d.points(points_gt, point_size=point_size, color=0x00ff88)
plot1 += k3d.points(points_1_aligned, point_size=point_size, color=0xff0000)
plot1 += k3d.text('Normalized CD: {:.4f}'.format(normalized_cd_1), position=[0, 0, 0], color=0x000000)
plot1.layout.width = '50%'

plot2 = k3d.plot(grid_visible=False)
plot2 += k3d.points(points_gt, point_size=point_size, color=0x00ff88)
plot2 += k3d.points(points_2_aligned, point_size=point_size, color=0x0000ff)
plot2 += k3d.text('Normalized CD: {:.4f}'.format(normalized_cd_2), position=[0, 0, 0], color=0x000000)
plot2.layout.width = '50%'

widgets.HBox([plot1, plot2])
